## import your modules needed

In [32]:
import requests as req
import time
import os
from dotenv import load_dotenv

In [33]:
load_dotenv()
sec_api = os.getenv("EDGAR_API_KEY")


In [34]:
company_tickers = ['UPST', 'KO', 'TSLA', 'INTC']

## use edgar api to search company using their ticker to fetch FR document 

In [35]:
def get_comp_fr_from_ticker(ticker):
    """
    Get the company financial reports for a given ticker symbol.
    
    Args:
        ticker (str): The ticker symbol of the company.
    
    Returns:
        object: filings of the company.
    """
    base_url = "https://api.sec-api.io"
    payload = {
        "query": {
            "query": {
                "query_string": {
                    "query": f"ticker:{ticker}"
                }
            },
            "from": "0",
            "size": "10",
            "sort": [
                {
                    "field": "filedAt",
                    "order": "desc"
                }
            ]
        },
        "options": {
            "skip_remap": True
        },
        "collection": "/edgar/financials"
    }
    print(f"Fetching company financial reports for {ticker}...")
    response = req.post(base_url, payload, headers={'Authorization': sec_api})
    
    if response.status_code == 200:
        comp_tenk_filing = response.json()
        return comp_tenk_filing
    else:
        raise Exception(f'Error fetching Company 10 k filings. Status code: {response.status_code}')

In [36]:
company_fr_filings = {}
for tkr in company_tickers:
    time.sleep(1)
    company_fr_filings[tkr] = get_comp_fr_from_ticker(tkr)

print(company_fr_filings)

Fetching company financial reports for UPST...
Fetching company financial reports for KO...
Fetching company financial reports for TSLA...
Fetching company financial reports for INTC...
{'UPST': {'total': {'value': 10000, 'relation': 'gte'}, 'query': {'from': 0, 'size': 200}, 'filings': [{'ticker': '', 'formType': '3', 'accessionNo': '0001641172-25-006057', 'cik': '2065631', 'companyNameLong': 'Hundorfean Cynthia D (Reporting)', 'companyName': 'Hundorfean Cynthia D', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/926617/000164117225006057/xslF345X02/ownership.xml', 'description': 'Form 3 - Initial statement of beneficial ownership of securities', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/2065631/000164117225006057/0001641172-25-006057.txt', 'filedAt': '2025-04-24T21:41:23-04:00', 'documentFormatFiles': [{'sequence': '1', 'size': '\xa0', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/926617/000164117225006057/xslF345X02/ownership.xml', 'description'

## instantiate  and create a gemini ai object using the provided api key

## create a LLM that model that iterate through the list of pdf files to analyze, syntesize, and summarize the pdf pages content